In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
class MyDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return self.x.size(0)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return (x.type(torch.double), y.type(torch.double))
    

def error(a,b):
    return (0.5*torch.mean((a-b)**2))

In [ ]:
import scipy.io as sc
import matplotlib.pyplot as plt
device = 'cuda:0'

In [6]:
def error_rms(a,b):
    return 0.5*np.sqrt(np.sum((a-b)**2)/a.size)
def NRMSE(predicted_value,target_value):
	MSE=np.mean(np.power((predicted_value-target_value),2))
	RMSE=np.sqrt(MSE)
	P=target_value.shape[0]
	NRMSE=RMSE/np.sum((np.max(target_value,1)-np.min(target_value,1))/P)
	return NRMSE

In [ ]:
# X = inputdata.T
# Y = targetdata.T

temp1 = sc.loadmat('X.mat')
temp2 = sc.loadmat('Y.mat')
X_train = torch.tensor(temp1['X']['train'][0][0].T).to(torch.double).to(device)
Y_train = torch.tensor(temp2['Y']['train'][0][0].T).to(torch.double).to(device)
X_test = torch.tensor(temp1['X']['test'][0][0].T).to(torch.double).to(device)
Y_test = torch.tensor(temp2['Y']['test'][0][0].T).to(torch.double).to(device)
del temp1
del temp2

# X_train = X_train[0:50000]
# X_test = X_test[0:20000]

In [8]:
rem_ind = []
for i in range(X_train.shape[1]):
    if X_train[:,i].mean().cpu().numpy()==0:
        rem_ind.append(i)
k = 0
for i in rem_ind:
    X_train = torch.cat((X_train[:,:i-k],X_train[:,i+1-k:]),1)
    k+=1
rem_ind = []
for i in range(X_test.shape[1]):
    if X_test[:,i].mean().cpu().numpy()==0:
        rem_ind.append(i)
k = 0
for i in rem_ind:
    X_test = torch.cat((X_test[:,:i-k],X_test[:,i+1-k:]),1)
    k+=1

In [9]:
MEAN = torch.mean(X_train, dim = 0)
STD = torch.std(X_train, dim = 0)
X_train = (X_train-MEAN)/STD
X_test = (X_test-MEAN)/STD


In [10]:
# model = model.to(torch.double)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NUM_EPOCHS = 90  
BATCH_SIZE = 128
MOMENTUM = 0.9
LR_DECAY = 0.0005
LR_INIT = 0.01
IMAGE_DIM = 227  
NUM_CLASSES = 1000  
DEVICE_IDS = [0]
INPUT_ROOT_DIR = 'alexnet_data_in'
TRAIN_IMG_DIR = 'alexnet_data_in/imagenet'
OUTPUT_DIR = 'alexnet_data_out'
LOG_DIR = OUTPUT_DIR + '/tblogs'  
CHECKPOINT_DIR = OUTPUT_DIR + '/models'  

os.makedirs(CHECKPOINT_DIR, exist_ok=True)


class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):

        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=1, padding='same'),  # (b x 96 x 55 x 55)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3
            # nn.MaxPool2d(kernel_size=2, stride=2),  # (b x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            # nn.MaxPool2d(kernel_size=2, stride=2),  # (b x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding='same'),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding='same'),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 1, 3, padding='same'),  # (b x 256 x 13 x 13)
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
        )
        # classifier is just a name for linear layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),#, inplace=True
            nn.Linear(in_features=(1*8*13), out_features=1024),#4096#256 * 6 * 6
            nn.ReLU(),
            nn.Dropout(p=0.5),#, inplace=True
            nn.Linear(in_features=1024, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=num_classes),
        )
        # self.init_bias()  # initialize bias

    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # nn.init.constant_(self.net[4].bias, 1)
        # nn.init.constant_(self.net[10].bias, 1)
        # nn.init.constant_(self.net[12].bias, 1)

    def forward(self, x):
        """
        Pass the input through the net.

        Args:
            x (Tensor): input tensor

        Returns:
            output (Tensor): output tensor
        """
        x = self.net(x)
        x = x.view(-1, 1*8*13)  # reduce the dimensions for linear layer input  #256 * 6 * 6
        return self.classifier(x)

In [12]:
X_train = X_train.reshape(-1,1,8,13).to(torch.float)
X_test = X_test.reshape(-1,1,8,13).to(torch.float)
Y_train = Y_train.to(torch.float)
Y_test = Y_test.to(torch.float)

In [ ]:
alexnet = AlexNet(num_classes=Y_train.shape[1]).to(device)


In [57]:
model = alexnet
torch.set_num_threads(1)
import time
# device = 'cpu'
device = 'cuda:0'
ss = torch.ones([1,1,8,13]).to(device)#.to(torch.float64)
model = model.to(device)
t1 = time.time()
model.forward(ss)
t2 = time.time()
print(t2-t1)

0.002073049545288086


In [17]:
# alexnet(torch.rand((100,1,8,13)).to('cuda:0')).shape

In [18]:
# alexnet.net(torch.rand((100,1,8,13)).to('cuda:0')).shape

In [19]:
# torch.save(alexnet.net,'alexnet_conv_2_8_13.pth')
# model = torch.load('alexnet_conv_2_8_13.pth')

In [ ]:

seed = torch.initial_seed()
print('Used seed : {}'.format(seed))

tbwriter = SummaryWriter(log_dir=LOG_DIR)
print('TensorboardX summary writer created')

alexnet = AlexNet(num_classes=Y_train.shape[1]).to(device)
alexnet = torch.nn.parallel.DataParallel(alexnet, device_ids=[0])#DEVICE_IDS

# dataset = datasets.ImageFolder(TRAIN_IMG_DIR, transforms.Compose([
#     # transforms.RandomResizedCrop(IMAGE_DIM, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
#     transforms.CenterCrop(IMAGE_DIM),
#     # transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ]))
# dataloader = data.DataLoader(
#     dataset,
#     shuffle=True,
#     pin_memory=True,
#     num_workers=8,
#     drop_last=True,
#     batch_size=BATCH_SIZE)

trainset = MyDataset(X_train, Y_train)
batch_size = 128#*4
dataloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
print('Dataloader created')
# alexnet = alexnet.to(torch.double)
optimizer = optim.Adam(params=alexnet.parameters(), lr=0.0001)
# optimizer = optim.SGD(
#     params=alexnet.parameters(),
#     lr=LR_INIT,
#     momentum=MOMENTUM,
#     weight_decay=LR_DECAY)
print('Optimizer created')

# multiply LR by 1 / 10 after every 30 epochs
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
print('LR Scheduler created')

# start training!!
print('Starting training...')
total_steps = 1
for epoch in range(NUM_EPOCHS):
    lr_scheduler.step()
    for imgs, classes in dataloader:
        imgs, classes = imgs.to(torch.float).to(device), classes.to(torch.float).to(device)

        # calculate the loss
        output = alexnet(imgs)
        loss = F.mse_loss(output, classes)#cross_entropy

        # update the parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # log the information and add to tensorboard
        if total_steps % 10 == 0:
            with torch.no_grad():
                _, preds = torch.max(output, 1)
                # accuracy = torch.sum(preds == classes)

                print('Epoch: {} \tStep: {} \tLoss: {:.4f} '#\tAcc: {}
                    .format(epoch + 1, total_steps, loss.item()))#, accuracy.item()
                tbwriter.add_scalar('loss', loss.item(), total_steps)
                # tbwriter.add_scalar('accuracy', accuracy.item(), total_steps)

        # print out gradient values and parameter average values
        # if total_steps % 100 == 0:
        #     with torch.no_grad():
        #         # print and save the grad of the parameters
        #         # also print and save parameter values
        #         print('*' * 10)
        #         for name, parameter in alexnet.named_parameters():
        #             if parameter.grad is not None:
        #                 avg_grad = torch.mean(parameter.grad)
        #                 print('\t{} - grad_avg: {}'.format(name, avg_grad))
        #                 tbwriter.add_scalar('grad_avg/{}'.format(name), avg_grad.item(), total_steps)
        #                 tbwriter.add_histogram('grad/{}'.format(name),
        #                         parameter.grad.cpu().numpy(), total_steps)
        #             if parameter.data is not None:
        #                 avg_weight = torch.mean(parameter.data)
        #                 print('\t{} - param_avg: {}'.format(name, avg_weight))
        #                 tbwriter.add_histogram('weight/{}'.format(name),
        #                         parameter.data.cpu().numpy(), total_steps)
        #                 tbwriter.add_scalar('weight_avg/{}'.format(name), avg_weight.item(), total_steps)

        total_steps += 1

    # save checkpoints
    checkpoint_path = os.path.join(CHECKPOINT_DIR, 'alexnet_states_e{}.pkl'.format(epoch + 1))
    state = {
        'epoch': epoch,
        'total_steps': total_steps,
        'optimizer': optimizer.state_dict(),
        'model': alexnet.state_dict(),
        'seed': seed,
    }
    torch.save(state, checkpoint_path)

Used seed : 93134069576400
TensorboardX summary writer created
AlexNet created
Dataloader created
Optimizer created
LR Scheduler created
Starting training...


c:\Users\kiano\.conda\envs\py38\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: 1 	Step: 10 	Loss: 0.0304 
Epoch: 1 	Step: 20 	Loss: 0.0289 
Epoch: 1 	Step: 30 	Loss: 0.0213 
Epoch: 1 	Step: 40 	Loss: 0.0157 
Epoch: 1 	Step: 50 	Loss: 0.0132 
Epoch: 1 	Step: 60 	Loss: 0.0147 
Epoch: 1 	Step: 70 	Loss: 0.0122 
Epoch: 1 	Step: 80 	Loss: 0.0116 
Epoch: 1 	Step: 90 	Loss: 0.0125 
Epoch: 1 	Step: 100 	Loss: 0.0125 
Epoch: 1 	Step: 110 	Loss: 0.0116 
Epoch: 1 	Step: 120 	Loss: 0.0106 
Epoch: 1 	Step: 130 	Loss: 0.0114 
Epoch: 1 	Step: 140 	Loss: 0.0097 
Epoch: 1 	Step: 150 	Loss: 0.0087 
Epoch: 1 	Step: 160 	Loss: 0.0100 
Epoch: 1 	Step: 170 	Loss: 0.0104 
Epoch: 1 	Step: 180 	Loss: 0.0097 
Epoch: 1 	Step: 190 	Loss: 0.0117 
Epoch: 1 	Step: 200 	Loss: 0.0092 
Epoch: 1 	Step: 210 	Loss: 0.0091 
Epoch: 1 	Step: 220 	Loss: 0.0076 
Epoch: 1 	Step: 230 	Loss: 0.0097 
Epoch: 1 	Step: 240 	Loss: 0.0090 
Epoch: 1 	Step: 250 	Loss: 0.0100 
Epoch: 1 	Step: 260 	Loss: 0.0088 
Epoch: 1 	Step: 270 	Loss: 0.0082 
Epoch: 1 	Step: 280 	Loss: 0.0092 
Epoch: 1 	Step: 290 	Loss: 0.

In [15]:
# alexnet = AlexNet(num_classes=Y_train.shape[1]).to(device)
# alexnet = torch.nn.parallel.DataParallel(alexnet, device_ids=[0])
# alexnet.load_state_dict(torch.load('alexnet_data_out/models/alexnet_states_e90.pkl')['model'])#state_dict()

<All keys matched successfully>

In [1]:
# torch.save(alexnet, 'alexnet_data_out/models/alexnet.pth')
# model = torch.load('alexnet_data_out/models/alexnet.pth')

In [ ]:
# alexnet(X_train[0:100]).shape

In [ ]:
# alexnet.net(X_train[0:100]).shape

In [ ]:
alexnet2 = AlexNet(num_classes=Y_train.shape[1]).to(device)
optimizer2 = optim.Adam(params=alexnet2.parameters(), lr=0.0001)
temp = alexnet2(torch.rand(50,1,8,13).to('cuda:0'))
loss = F.mse_loss(output, torch.rand(50,6).to('cuda:0'))

In [ ]:
optimizer2.zero_grad()


In [ ]:
loss.backward()

In [ ]:
# torch.save(model, 'GT2RBFnn_justCenter_20n_11s.pt')
# model = torch.load('GT2RBFnn_justCenter_20n_11s.pt')

In [ ]:
def Train(model, X, Y, epochs=1000, batch_size = 50, loss_func = nn.MSELoss(), lr = 0.01):
    trainset = MyDataset(X, Y)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
        
    optimiser = torch.optim.Adam(model.parameters(), lr=lr)
        
    tqdm_t = trange(epochs, desc='MSE:', leave=True)
    for epoch in tqdm_t:
        for x_batch, y_batch in trainloader:
            optimiser.zero_grad()
            y_hat = model.forward(x_batch)
            loss = loss_func(y_hat, y_batch)#.type(torch.double)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Clip gradients

            # for p in self.parameters():
            #     p.data.clamp_(-100, 100)

            optimiser.step()

            #self.W = torch.linalg.pinv(self.phi(x_batch)).matmul(y_batch)
        tqdm_t.set_description("MSE: (%e)" % loss)
        tqdm_t.refresh()

In [ ]:
# model.mu = nn.Parameter(model.mu)
X_train = X_train.reshape([1,13,8])

In [ ]:
X = X_test
Y = Y_test
Y_hat = alexnet(X[200*0:200*(0+1)]).cpu().detach().numpy()
for i in range(1,int(X.shape[0]/200)):
    Y_hat = np.concatenate((Y_hat,alexnet(X[200*i:200*(i+1)]).cpu().detach().numpy()))

In [ ]:
def PCC(predicted_output, desired_output):
    mean_predicted = torch.mean(predicted_output)
    mean_desired = torch.mean(desired_output)

    numerator = torch.sum((predicted_output - mean_predicted) * (desired_output - mean_desired))
    denominator_predicted = torch.sqrt(torch.sum((predicted_output - mean_predicted)**2))
    denominator_desired = torch.sqrt(torch.sum((desired_output - mean_desired)**2))
    pcc = numerator / (denominator_predicted * denominator_desired)

    # Print the result
    print("Pearson Correlation Coefficient:", pcc.item())

In [38]:
PCC(torch.tensor(Y_hat).to(device), Y[0:Y_hat.shape[0]])

Pearson Correlation Coefficient: 0.8899387121200562


In [39]:
NRMSE(Y_hat, Y[0:Y_hat.shape[0]].cpu().numpy())

0.15947026